In [2]:
# %%
import os
import sys

# Adjust the path to import se2_function
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)

from py_utils.path_aux_utils import *
# check if the required packages are installed and install them if not
pre_required_install_check()

import sys
import subprocess
import pkg_resources

# Check if spatialmath-python is installed, if not install it
package_name = 'spatialmath-python'
try:
    pkg_resources.get_distribution(package_name)
except pkg_resources.DistributionNotFound:
    print(f"{package_name} is not installed. Installing now...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])

import matplotlib.pyplot as plt
from numpy.random import default_rng
import math
import numpy as np
from typing import Optional
from spatialmath import SE2
import os

# Adjust the path to import se2_function
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)

from py_utils.path_loader_utils import *
from py_utils.se2_function import *
from py_utils.path_geom_utils import * 
from py_utils.path_aux_utils import *
from py_utils.plot_helpers import path_merger_plot, path_merger_plot_cpp
from py_utils.path_merger_utils import PathMergerOptions, path_merger_wrapper
from py_utils.PathMerger import PathMerger
from sim_path_merger import path_pre_processing
# %%
BUILD_PATH = '/opt/imagry/aidriver_new/cmake-bin/modules/planner/tools/pybind11_projects/py_path_merger'

if BUILD_PATH not in sys.path:
    sys.path.append(BUILD_PATH)

import numpy as np
import libpy_aidriver_path_merger as pm

# Set the path to the data directory 
# IMPORTANT: In general, the data should not be stored in /opt/imagry/aidriver_new
INPUT_DATA_PATH = '/opt/imagry/aidriver_new/modules/planner/tools/pybind11_projects/py_path_merger/data'


In [ ]:

# %%
#%% Path merger example
# def test_path_merger_wrapper():
#  Example code to test the path merger wrapper
    
# Generate data in world coordinates for two consecutive frames
x1 = np.hstack((np.arange(0,1,0.1), np.arange(1.05,20,1)))
y1_ = lambda x1: 9 * (x1 ** 0.4)
X_1_world, _, _, p1_world, p1_p1 = path_pre_processing(x1, y1_)

x2 = np.hstack((np.arange(5,6,0.5), np.arange(6.05,30,1)))
y2_ = lambda x2: 2.5 + 9 * (x2 ** 0.3)
X_2_world, X_now_world, _, p2_world, p2_p2  = path_pre_processing(x2, y2_)

lm1 = 0.5 #smoothing lambda
lm2 = 0.5 #derivative constraint lambda
w_inc = 5 #weighing of p2 before i2 (larger value means faster decay)

# print(dir(libpath_merger))

# Call the C++ function to get the merged path
# Test the Class version
# instantiate PathMerger class
path_merger = PathMerger(tau_1_ms=0.2, N2=30, lambda_1=lm1, lambda_2=lm2, w=w_inc, spline_degree=3, DEBUG_MODE=True)

# feed it the first path p1_p1
p1_p1_merged = path_merger.merge_with_new_path(p1_p1, X_1_world, None)

# feed it with the second path p2_p2
p2_p2_merged = path_merger.merge_with_new_path(p2_p2, X_2_world, X_now_world)

# plot the merged path
p2_merged_world = apply_se2_transform(X_2_world, p2_p2_merged)
p1_p2 = apply_se2_transform(X_2_world.inv(), p1_world)

history = path_merger.history
i0, i1, i12, i2 = history.iloc[-1][3:]
path_merger_plot(X_1_world, X_2_world, X_now_world, p1_world, p2_world, p2_merged_world, i0, i1, i12, i2, p2_p2_merged, p1_p2, p2_p2)
plt.savefig('modules/planner/tools/pybind11_projects/py_path_merger/src/python/figures/'+'class_output_figure.png')



In [ ]:

### Now, run the C++ version of the path merger
# Initialize the TemporalPathMerger
merger = pm.TemporalPathMerger(tau=0.2, N2=30, lm1=0.5, lm2=0.5, w_inc=5, spline_degree=3, step_size=0.1)
        
p1_p1_merged_path_cpp = merger.merge_paths(
    input_path_ego=p1_p1, 
    car_pose_img=X_1_world.A, 
    car_pose_now=np.identity(3), 
    target_speed_mps=2.7
)
p2_p2_merged_path_cpp = merger.merge_paths(
    input_path_ego=p2_p2, 
    car_pose_img=X_2_world.A, 
    car_pose_now=X_now_world.A, 
    target_speed_mps=2.7
)

#%% Plots
p2_p2_merged_cpp = np.array(p2_p2_merged_path_cpp)
p2_p2_world_merged_cpp = apply_se2_transform(X_2_world, p2_p2_merged_cpp)

path_merger_plot_cpp(X_1_world, X_2_world, X_now_world, p1_world, p2_world, p2_p2_world_merged_cpp)

# Providing specific arguments
plt.savefig('modules/planner/tools/pybind11_projects/py_path_merger/src/python/figures/'+'cpp_output_figure.png')

    
# %%  


